In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import re

In [2]:
#크롤링을 통해 책의 고유 ID를 수집
num = 1
itemids = []
i = 1

#책 id를 추출하는 정규식
compiler1 = re.compile('/Product/Goods/\d{5,8}')
compiler2 = re.compile('\d{5,8}')

while True:
    url = 'http://www.yes24.com/searchcorner/Search?keywordAd=&keyword=&query=%c0%ce%b0%f8%c1%f6%b4%c9&domain=BOOK&page_size=120&PageNumber={}&scode=012'
    page = requests.get(url .format(num))
    soup = BeautifulSoup(page.text, 'html.parser')
    
    bookInfo = soup.select('p.goods_name a')
    if len(bookInfo) <= 0:
        print('Done!')
        break
    
    for info in bookInfo:
        try:
            itemid=compiler1.search(str(info)).group()
            itemids.append(compiler2.search(itemid).group())
        except AttributeError:
            pass

    num += 1
print('총 개수: ', len(itemids))

Done!
총 개수:  807


In [60]:
# DataFrame 생성
columns = ['제목','부제', '저자', '분야', '출판일', '출판사', '가격', '목차', '소개', '출판사제공'] #데이터 프레임의 컬럼명 
dataFrame = pd.DataFrame(columns=columns) #dataFrame을 만들고 지정한 column명으로 설정

In [61]:
# 책의 상세 페이지에 접근하여 데이터 크롤링
n = 0 #dataframe index 초기화
year = 2012  #검색 기간
for i in itemids:
    url = 'http://www.yes24.com/Product/Goods/{}?scode=032&OzSrank=1'
    page = requests.get(url .format(i))
    soup = BeautifulSoup(page.text, 'html.parser')
    
    #출간일
    try:
        date = soup.find('span', attrs={'class':'gd_date'}).text
        date = re.sub('년|월|일|\ ', '', date) #한글 단위를 모두 제거
        if int(date[:4]) < year:
            continue
    except AttributeError:
        continue
    
    
    title = soup.find('h2', attrs={'class':'gd_name'}).text #제목을 가져온다.
    if '세트' in title:
        continue
    try:
        sub_title = soup.find('h3', attrs={'class':'gd_nameE'}).text  #부제목을 가져온다.
    except AttributeError:
        sub_title = None
        
    try:
        author = soup.select('span.gd_auth a')[0].text #저자 정보
    except IndexError:
        try:
            author = soup.select('span.gd_auth')[0].text #저자 정보
            author = re.sub('\\r|\\n|\ ', '', author)
        except IndexError:
            author = None
            
    feild = soup.select('div.gd_infoSet a')[1].text  #분야를 가져온다.

    

    publisher = soup.select('span.gd_pub')[0].text #출판사

    #가격
    price = soup.find('em', attrs={'class':'yes_m'}).text
    price = re.sub(',|원', '', price)
    
    #목차
    try:
        contents = soup.select('textarea.txtContentText')[1].text
    except IndexError:
        try:
            contents = soup.select('textarea.txtContentText')[0].text
        except IndexError:
            continue
    
    introduction = soup.select('textarea.txtContentText')[0].text #책소개
    
    #출판사 소개
    try:
        introduction_2 = soup.select('div#infoset_pubReivew')[0].text 
        introduction_2 = re.sub('출판사 리뷰|\\n|펼쳐보기접어보기', ' ', introduction_2) #불용어 제거
    except IndexError:
        introduction_2 = None
    
    #['제목','부제', '저자', '분야', '출판일', '출판사', '가격', '목차', '소개', '출판사제공']
    dataFrame.loc[n] = [title, sub_title, author, feild, date, publisher, price, contents, introduction, introduction_2]
    n+= 1;

print('데이터 개수: ', len(dataFrame))
print('Done!')

Done!


In [65]:
dataFrame.to_csv(r'AI_book_data.csv', index=False, sep=',', encoding='utf-8')

In [62]:
len(dataFrame)

599